In [14]:
import requests
import json, os
import logging
from time import sleep
from datetime import datetime

# data AROME from opendatasoft

In [13]:
# ============
#  Parameters
# ============
dataset = 'arome-0025-sp1_sp2'
commune = 'Marseille'
lat_lon = (43.3, 5.4)

save_dir = 'data'

wait_hours = 3

In [3]:
logging.basicConfig(filename=os.path.join(save_dir, 'log.txt'),
                    format='%(asctime)s - %(levelname)s  %(message)s',
                    level=logging.INFO)

In [11]:
try:
    os.mkdir(save_dir)
    print('mkdir', save_dir)
except FileExistsError:
    pass

In [12]:
logging.info('start script')

In [20]:
def query():
    url = "https://public.opendatasoft.com/api/records/1.0/search/"

    lat, lon = lat_lon
    params = {'dataset':dataset,
              'rows':100,
              'facet':['communes', 'code_commune'],
              'refine.commune':commune,
              'geofilter.distance':f"{lat:.2f}, {lon:.2f}, 100",
              'sort':'forecast'}

    r = requests.get(url, params=params)

    if r.status_code != 200:
        logging.error('query error')
        print('query error')
        print(r.text)
    else:

        data = r.json()
        records = data['records']
        fields = [r['fields'] for r in records]

        times = [(f['timestamp'], f['forecast']) for f in fields]
        date_of_the_run = min(times, key=lambda x:x[1])[0]

        filename = os.path.join(save_dir, f'{date_of_the_run}.json')
        with open(filename, 'w') as outfile:
            json.dump(data, outfile)

        print(' > save:', filename)
        logging.info(f'save - run:{date_of_the_run} {len(times)} points')

In [21]:
while True:
    query()
    print(datetime.now(), "waiting...")
    sleep(wait_hours *60*60)

 > save: data/2020-07-26T03:00:00+00:00.json
2020-07-26 15:38:19.938620 waiting...
 > save: data/2020-07-26T03:00:00+00:00.json
2020-07-26 15:38:27.835877 waiting...


KeyboardInterrupt: 